In [41]:
import numpy as np
import pandas as pd
from ast import literal_eval

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [42]:
import pandas as pd
import numpy as np

merged_df = pd.read_csv("/Users/Shared/최종선_교수님/Face_skin_disease/감성분석/merged_output.csv")
merged_df.head()
df = merged_df
df


,product_name,reviewer_id,skin_type,skin_tone,skin_concern_1,skin_concern_2,rating,review_date,review,tokens,Ntoken_review,pred
0,닥터지 더모이스처 배리어D 인텐스 크림,리리578,건성,봄웜톤,모공,미백,5,2025.07.24,닥터지 더모이스처 배리어 인텐스크림 제가 이 제품을 사용하고 이렇게 오랜기간...,"[('닥터', 'NNP'), ('지', 'NNB'), ('더', 'MAG'), ('...","['닥터', '이스', '리어', '스크림', '제가', '제품', '사용', '기...",0
1,닥터지 더모이스처 배리어D 인텐스 크림,김츄츄,건성,겨울쿨톤,각질,아토피,5,2025.07.27,악건성인데 화장하기 전에 무조건 바르는 크림입니다 화장 잘 뜨고 각질 많이 일어나...,"[('악건성', 'NNG'), ('이', 'VCP'), ('ㄴ데', 'EC'), (...","['악건성', '화장', '크림', '화장', '각질', '건성', '정말', '정...",0
2,닥터지 더모이스처 배리어D 인텐스 크림,화사한웜코덕,NaN,NaN,NaN,NaN,5,2025.05.31,현직메컵아티스트입니다이거 안맞는 분 한번도 못봤어용 기초할때 선크림 직전에 발라주고...,"[('현직', 'NNG'), ('메', 'NNG'), ('컵', 'NNG'), ('...","['현직', '아티스트', '기초', '선크림', '직전', '흡수', '선크림',...",0
3,닥터지 더모이스처 배리어D 인텐스 크림,앙정,건성,봄웜톤,모공,잡티,5,2025.06.22,보습 필요할때 사용하고 있어요가볍게 바르기 좋고 보습을 필요로할때는 듬뿍 발라주고 있어요,"[('보습', 'NNG'), ('필요', 'NNG'), ('하', 'XSV'), (...","['보습', '필요', '사용', '보습', '필요']",1
4,닥터지 더모이스처 배리어D 인텐스 크림,mimilike,건성,웜톤,각질,주름,4,2025.07.11,다른 크림들과 비교해서 확실히 가려움에 효과가 있어요 그래서 저는 아토피 올라오는...,"[('다른', 'MM'), ('크림', 'NNG'), ('들', 'XSN'), ('...","['크림', '비교', '가려움', '효과', '아토피', '부분', '세라마이드'...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
1089,에스트라 아토베리어 365크림,볼빵빵춘식이,트러블성,가을웜톤,각질,모공,5,2024.02.09,가격 신상티켓팅 라이브에서 신청해서 받았습니다 월 일 기준 가격 원 월 일...,"[('가격', 'NNG'), ('신상티켓팅', 'NA'), ('라이브', 'NNP'...","['가격', '라이브', '신청', '기준', '가격', '세일', '중인', '가...",1
1090,에스트라 아토베리어 365크림,로즈니,복합성,봄웜톤,모공,미백,5,2024.02.09,신상티켓팅 체험단 민감성 수부지 봄 웜톤 밝은 호에스트라 그리고 아토...,"[('신상티켓팅', 'NA'), ('체험단', 'NNG'), ('민감성', 'NNG...","['체험단', '민감성', '수부지', '아토', '베리', '소문', '피부', ...",0
1091,에스트라 아토베리어 365크림,예민마루,민감성,쿨톤,민감성,트러블,5,2024.02.09,기존 세대 제품을 사용해보긴 했었는데 제 피부에는 안맞았었어요 캡슐 알갱이가 자...,"[('기존', 'NNG'), ('세대', 'NNG'), ('제품', 'NNG'), ...","['기존', '세대', '제품', '사용', '피부', '캡슐', '알갱이', '자...",0
1092,에스트라 아토베리어 365크림,꾸안꾸꾸꾹,중성,쿨톤,각질,블랙헤드,5,2024.02.09,로션이나크림은 기능성보다 보습에 충실한 제품좋아합니다그래서 피지오겔 일리윤 제로이드...,"[('로션', 'NNP'), ('이나', 'JC'), ('크림', 'NNP'), (...","['로션', '크림', '기능', '보습', '충실', '제품', '일리', '제로...",0


In [43]:
# Surprise (협업 필터링용)
try:
    from surprise import Dataset, Reader, SVD
    HAS_SURPRISE = True
except Exception:
    HAS_SURPRISE = False
    print("⚠️ Surprise가 설치되지 않아 협업 필터링(SVD)을 건너뜁니다. 필요시 %pip install scikit-surprise 실행하세요.")

### 데이터 준비 

In [44]:
import pandas as pd
from ast import literal_eval

# 0) 필수 컬럼 체크
need = {"reviewer_id","product_name","rating","review"}
miss = need - set(df.columns)
if miss:
    raise ValueError(f"필수 컬럼 없음: {miss}")

# 1) 컬럼명만 변경 (새 컬럼 생성 X)
rename_map = {
    "reviewer_id": "user_id",
    "product_name": "item_id",
    "review": "text",        # review -> text 로 이름만 변경
    "rating": "rating_aug",  # rating -> rating_aug 로 이름만 변경
}
conflict = (set(rename_map.values()) & set(df.columns)) - set(rename_map.keys())
if conflict:
    raise ValueError(f"이미 존재하는 컬럼명과 충돌: {conflict}")
df.rename(columns=rename_map, inplace=True)

# 2) 'text' 컬럼 내용을 tokens로 강화 (동일 컬럼명 덮어쓰기, 새 컬럼 생성 없음)
def join_tokens(x):
    if isinstance(x, list):
        return " ".join(map(str, x))
    if pd.isna(x):
        return ""
    # 문자열로 저장된 리스트 형태("[... ]")가 있을 수 있어 간단 복원 시도
    s = str(x).strip()
    if s.startswith("[") and s.endswith("]"):
        try:
            parsed = literal_eval(s)
            if isinstance(parsed, list):
                return " ".join(map(str, parsed))
        except Exception:
            pass
    return s

if "tokens" in df.columns:
    df["text"] = df["text"].fillna("").astype(str) + " " + df["tokens"].apply(join_tokens).fillna("")
else:
    df["text"] = df["text"].fillna("").astype(str)

# 3) 'rating_aug' 컬럼에 pred 반영 (동일 컬럼명 덮어쓰기, 새 컬럼 생성 없음)
alpha = 0.5
pred_series = df["pred"] if "pred" in df.columns else 0
df["rating_aug"] = (
    pd.to_numeric(df["rating_aug"], errors="coerce").fillna(0.5)
    + alpha * pd.to_numeric(pred_series, errors="coerce").fillna(0.0)
).clip(0.5, 5.0)

display(df.head(3))


,item_id,user_id,skin_type,skin_tone,skin_concern_1,skin_concern_2,rating_aug,review_date,text,tokens,Ntoken_review,pred
0,닥터지 더모이스처 배리어D 인텐스 크림,리리578,건성,봄웜톤,모공,미백,5.0,2025.07.24,닥터지 더모이스처 배리어 인텐스크림 제가 이 제품을 사용하고 이렇게 오랜기간...,"[('닥터', 'NNP'), ('지', 'NNB'), ('더', 'MAG'), ('...","['닥터', '이스', '리어', '스크림', '제가', '제품', '사용', '기...",0
1,닥터지 더모이스처 배리어D 인텐스 크림,김츄츄,건성,겨울쿨톤,각질,아토피,5.0,2025.07.27,악건성인데 화장하기 전에 무조건 바르는 크림입니다 화장 잘 뜨고 각질 많이 일어나...,"[('악건성', 'NNG'), ('이', 'VCP'), ('ㄴ데', 'EC'), (...","['악건성', '화장', '크림', '화장', '각질', '건성', '정말', '정...",0
2,닥터지 더모이스처 배리어D 인텐스 크림,화사한웜코덕,NaN,NaN,NaN,NaN,5.0,2025.05.31,현직메컵아티스트입니다이거 안맞는 분 한번도 못봤어용 기초할때 선크림 직전에 발라주고...,"[('현직', 'NNG'), ('메', 'NNG'), ('컵', 'NNG'), ('...","['현직', '아티스트', '기초', '선크림', '직전', '흡수', '선크림',...",0


KoBERT 임베딩 (아이템 벡터)

콘텐츠 추천 (KoBERT)

(옵션) 협업 필털이 SVD + 추천

### 피부 적합도 + 하이브리드 & 통합벡터